<a href="https://colab.research.google.com/github/Lorenzo2812/Master-Thesis-Repository/blob/main/Draft_Image_Segmentation_Work_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMAGE SEGMENTATION TASK: ISIC 2017 DATASET

### IMAGE IMPORTING, PRE-PROCESSING AND DATA AUGMENTATION

VISUALIZING GPU DETAILS

In [ ]:
!nvidia-smi

LIBRARIES TO BE INSTALLED

In [ ]:
#FOR DATA AUGMENTATION
!pip install imgaug
#FOR HYPERPARAMETERS TUNING
!pip install -q -U keras-tuner

LIBRARIES TO BE IMPORTED

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import RMSprop
from sklearn.model_selection import GridSearchCV
from keras import layers,models
import keras_tuner as kt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import EarlyStopping
import keras
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns
from sklearn.metrics import classification_report
import pandas as pd
from keras.utils import plot_model
import pydot
import graphviz
import pydotplus
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from keras.utils import to_categorical
import shutil
from keras.models import Model
from keras.layers import Input, Conv2D, GlobalAveragePooling2D, Dense
from keras.applications.vgg16 import VGG16
from sklearn.metrics import accuracy_score
from google.colab import drive
import time
import imgaug.augmenters as iaa
import imageio
import tensorflow as tf
from tensorflow import keras
from keras import backend as K
from keras_tuner import HyperModel, Hyperband, Objective
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from keras.models import load_model
from imgaug.augmentables.segmaps import SegmentationMapsOnImage

CONNECTING TO GOOGLE DRIVE

In [ ]:
drive.mount('/content/drive')   #capability to navigate through my drive folders


DATA UPLOADING, RESIZING, CONVERTING TO GRAYSCALE AND NORMALIZING

In [ ]:
# pre-processing the data: all folders of data imported from google drive: /content/drive
train_data_path = '/content/drive/MyDrive/TESI SU SEGMENTAZIONE IMMAGINI (ISIC 2017)/ISIC-2017_Training_Data'
train_mask_data_path = '/content/drive/MyDrive/TESI SU SEGMENTAZIONE IMMAGINI (ISIC 2017)/ISIC-2017_Training_Part1_GroundTruth'
validation_data_path = '/content/drive/MyDrive/TESI SU SEGMENTAZIONE IMMAGINI (ISIC 2017)/ISIC-2017_Validation_Data'
validation_mask_data_path = '/content/drive/MyDrive/TESI SU SEGMENTAZIONE IMMAGINI (ISIC 2017)/ISIC-2017_Validation_Part1_GroundTruth'
test_data_path = '/content/drive/MyDrive/TESI SU SEGMENTAZIONE IMMAGINI (ISIC 2017)/ISIC-2017_Test_Data'
test_mask_data_path = '/content/drive/MyDrive/TESI SU SEGMENTAZIONE IMMAGINI (ISIC 2017)/ISIC-2017_Test_Part1_GroundTruth'
img_size = 256  #common image size I want to use: since higher resolution can lead to better performance this size could be more appropriate than 64

def create_data(path): #DATA PRE-PROCESS CUSTOMIZED FUNCTION
    data = [] #this list will store the output data
    error_count = 0
    total_time = 0
    num_images = 0

    image_files = sorted(os.listdir(path))

    for img in image_files:
        start_time = time.time()  #I want to monitor the time I will need to read and process my data
        try:
            img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)  # images to be read in grayscale
            if img_arr is None:
                print(f"Failed to load image: {os.path.join(path, img)}")
                error_count += 1
                continue
            resized_arr = cv2.resize(img_arr, (img_size, img_size))  # image resizing process
            data.append(resized_arr)
            num_images += 1
            print(f"Image {img} loaded successfully.")
        except Exception as e:
            print(f"Error processing image: {os.path.join(path, img)} - {e}")
            error_count += 1
        end_time = time.time()
        total_time += (end_time - start_time)

    print(f"Number of images loaded: {len(data)}")
    print(f"Number of images with errors: {error_count}")
    print(f"Total time taken: {total_time:.2f} seconds")
    if num_images > 0:
        print(f"Average time per image: {total_time / num_images:.2f} seconds")   #average time image loading

    return np.array(data) / 255.0  # normalizing my data after converting the list storing it into and array: I obtain that every pixel has a value between 0 and 1

#applying the function on all my data
train_data = create_data(train_data_path)
train_mask = create_data(train_mask_data_path)
validation_data = create_data(validation_data_path)
validation_mask = create_data(validation_mask_data_path)
test_data = create_data(test_data_path)
test_mask = create_data(test_mask_data_path)


In [ ]:
#Nr of images for set of data downloaded
print(f"Number of elements in train_data: {len(train_data)}")
print(f"Number of elements in train_mask: {len(train_mask)}")
print(f"Number of elements in validation_data: {len(validation_data)}")
print(f"Number of elements in validation_mask: {len(validation_mask)}")
print(f"Number of elements in test_data: {len(test_data)}")
print(f"Number of elements in test_mask: {len(test_mask)}")

SAVING ARRAYS RELATED TO THE IMAGES LOADED AND PRE-PROCESSED

In [ ]:
#Saving all the pre-processed images
np.save('/content/drive/MyDrive/TESI SU SEGMENTAZIONE IMMAGINI (ISIC 2017)/train_data.npy', train_data)
np.save('/content/drive/MyDrive/TESI SU SEGMENTAZIONE IMMAGINI (ISIC 2017)/train_mask.npy', train_mask)
np.save('/content/drive/MyDrive/TESI SU SEGMENTAZIONE IMMAGINI (ISIC 2017)/validation_data.npy', validation_data)
np.save('/content/drive/MyDrive/TESI SU SEGMENTAZIONE IMMAGINI (ISIC 2017)/validation_mask.npy', validation_mask)
np.save('/content/drive/MyDrive/TESI SU SEGMENTAZIONE IMMAGINI (ISIC 2017)/test_data.npy', test_data)
np.save('/content/drive/MyDrive/TESI SU SEGMENTAZIONE IMMAGINI (ISIC 2017)/test_mask.npy', test_mask)

LOADING THE ARRAYS SAVED BEFORE: THESE PROCESS CAN BE DONE TO AVOID APPLYING THE CREATE DATA FUNCTION EVERY TIME A NEW SESSION BEGINS

In [ ]:
# Load the arrays saved: to skip previous step for next session
train_data=np.load('/content/drive/MyDrive/TESI SU SEGMENTAZIONE IMMAGINI (ISIC 2017)/train_data.npy')
train_mask=np.load('/content/drive/MyDrive/TESI SU SEGMENTAZIONE IMMAGINI (ISIC 2017)/train_mask.npy')
validation_data = np.load('/content/drive/MyDrive/TESI SU SEGMENTAZIONE IMMAGINI (ISIC 2017)/validation_data.npy')
validation_mask = np.load('/content/drive/MyDrive/TESI SU SEGMENTAZIONE IMMAGINI (ISIC 2017)/validation_mask.npy')
test_data = np.load('/content/drive/MyDrive/TESI SU SEGMENTAZIONE IMMAGINI (ISIC 2017)/test_data.npy')
test_mask = np.load('/content/drive/MyDrive/TESI SU SEGMENTAZIONE IMMAGINI (ISIC 2017)/test_mask.npy')

EXAMPLE OF GRAYSCALE IMAGES AND SEGMENTATION MASKS LOADED

In [ ]:
#shape of array with train data
train_data.shape

In [ ]:
#Plotting an image of example
plt.figure(figsize=(3, 3))
plt.imshow(train_data[160], cmap='gray')
plt.title('Example Image')
plt.axis('off')
plt.show()

In [ ]:
#Plotting a mask of example
plt.figure(figsize=(3, 3))
plt.imshow(train_mask[160], cmap='gray')
plt.title('Example Mask')
plt.axis('off')
plt.show()

DATA AUGMENTATION: TO EXPAND THE DATASET, TRAIN DATA HAS BEEN AUGMENTED BY 5 TIMES, APPLYING DIFFERENT TRANFORMATIONS TO ORIGINAL DATA

In [ ]:
#PURPOSE: INCREASING VARIABILITY AND REDUCING OVERFITTING BY INTRODUCING TRANSFORMED VERSIONS OF MY TRAINING DATA TO EXPAND MY DATASET

#Defining the augmentation sequence
seq = iaa.Sequential([
    iaa.Fliplr(0.5),  # horizontal flips(50% images)
    iaa.Crop(percent=(0, 0.1)),  # random crops  (frpm 0 to 10% of the image)
    iaa.LinearContrast((0.75, 1.5)),  # contrast changes
    iaa.AdditiveGaussianNoise(scale=(0, 0.05*255)),  # add gaussian noise to simulate imperfections (from 0 to 5% of the pixel)
    iaa.Multiply((0.8, 1.2)),  # brightness changes
    iaa.Affine(
        rotate=(-25, 25),  # rotate by -25 to +25 degrees
        shear=(-8, 8)  # shear by -8 to +8 degrees (distorsion along the axis)
    )
])

# Path to my dataset
input_images_dir = train_data_path
input_masks_dir = train_mask_data_path
output_images_dir = '/content/drive/MyDrive/TESI SU SEGMENTAZIONE IMMAGINI (ISIC 2017)/ISIC-2017_Augmented_Data'           #PATHS WHERE IMAGES AND MASKS AUGMENTED WILL BE SAVED
output_masks_dir = '/content/drive/MyDrive/TESI SU SEGMENTAZIONE IMMAGINI (ISIC 2017)/ISIC-2017_Augmented_Masks'

# Number of augmented images to generate per original image: augmenting the dataset by 5 times: to every image corresponds 5 augmented images
num_augmented_images = 5

# Create output directories to store augmented data
os.makedirs(output_images_dir, exist_ok=True)
os.makedirs(output_masks_dir, exist_ok=True)

# Loop over all images in the dataset
for filename in os.listdir(input_images_dir):
    if filename.endswith('.jpg') or filename.endswith('.png'): #both if i've jpg and png
        image = imageio.imread(os.path.join(input_images_dir, filename))

        for i in range(num_augmented_images):
            image_aug = seq(image=image) #applying function with transformation sequence

            # Save augmented images
            output_image_path = os.path.join(output_images_dir, f'aug_{i}_{filename}')
            imageio.imwrite(output_image_path, image_aug)
            print(f'{i} Image {filename} augmented successfully as {output_image_path}')

#same process for masks
for filename in os.listdir(input_masks_dir):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        # Read the mask
        mask = imageio.imread(os.path.join(input_masks_dir, filename))

        # Wrap the mask in SegmentationMapsOnImage
        segmap = SegmentationMapsOnImage(mask, shape=mask.shape) #specific function for masks: helps mantaining the characteristics of these specific kind of images (which have also discrete values for diff regions and not continuous like regular color images)

        for i in range(num_augmented_images):

            segmap_aug = seq(segmentation_maps=segmap) #applying function with transformation sequence

            # Extract the augmented mask
            mask_aug = segmap_aug.get_arr()

            # Save the augmented mask
            output_mask_path = os.path.join(output_masks_dir, f'aug_{i}_{filename}')
            imageio.imwrite(output_mask_path, mask_aug)
            print(f'{i} Mask {filename} augmented successfully as {output_mask_path}')

print("Data augmentation for image segmentation completed!")



APPLYING ALL THE PRE-PROCESSING STEPS TO AUGMENTED IMAGES AND MASKS, LIKE I DID WITH TRAINING DATA

In [ ]:
#CREATE DATA FOR AUGMENTED DATA
output_images_dir = '/content/drive/MyDrive/TESI SU SEGMENTAZIONE IMMAGINI (ISIC 2017)/ISIC-2017_Augmented_Data'
augmented_images=create_data(output_images_dir)

In [ ]:
#CREATE DATA FOR AUGMENTED MASKS
output_masks_dir = '/content/drive/MyDrive/TESI SU SEGMENTAZIONE IMMAGINI (ISIC 2017)/ISIC-2017_Augmented_Masks'
augmented_masks=create_data(output_masks_dir)

In [ ]:
#SAVING THE ARRAYS
np.save('/content/drive/MyDrive/TESI SU SEGMENTAZIONE IMMAGINI (ISIC 2017)/augmented_images.npy', augmented_images)
np.save('/content/drive/MyDrive/TESI SU SEGMENTAZIONE IMMAGINI (ISIC 2017)/augmented_masks.npy', augmented_masks)

In [ ]:
#LOADING THE ARRAYS SAVED: TO SKIP THE PRE-PROCESSING STEPS EVEN FOR MY AUGMENTED DATA
augmented_images=np.load('/content/drive/MyDrive/TESI SU SEGMENTAZIONE IMMAGINI (ISIC 2017)/augmented_images.npy')
augmented_masks=np.load('/content/drive/MyDrive/TESI SU SEGMENTAZIONE IMMAGINI (ISIC 2017)/augmented_masks.npy')

In [ ]:
#PLOTTING AN AUGMENTED IMAGE: AN EXAMPLE
plt.figure(figsize=(3, 3))
plt.imshow(augmented_images[4000], cmap='gray')
plt.title('Example Image: Augmented')
plt.axis('off')
plt.show()

In [ ]:
#PLOTTING AN AUGMENTED MASK: AN EXAMPLE
plt.figure(figsize=(3, 3))
plt.imshow(augmented_masks[0], cmap='gray')
plt.title('Example Mask: Augmented')
plt.axis('off')
plt.show()

COMBINING TRAINING AND AUGMENTED DATA IN THE DATASET:  A NEW TRAINING SET 6 TIMES THE SIZE OF THE PREVIOUS ONE

In [ ]:
train_data_images = np.concatenate((train_data, augmented_images), axis=0)

train_data_masks = np.concatenate((train_mask, augmented_masks), axis=0)

print("Combined data images contains", len(train_data_images), "images.")

print("Combined data masks contains", len(train_data_masks), "masks.")

In [ ]:
#EXPANDING ARRAYS
train_data_images = np.expand_dims(train_data_images, axis=-1)
train_data_masks = np.expand_dims(train_data_masks, axis=-1)
validation_data= np.expand_dims(validation_data, axis=-1)
validation_mask = np.expand_dims(validation_mask, axis=-1)
test_data = np.expand_dims(test_data, axis=-1)
test_mask = np.expand_dims(test_mask, axis=-1)

In [ ]:
#NEW TRAIN DATA SIZE (CONTAINING EVEN AUGMENTED ONES)
train_data_images.shape
train_data_masks.shape

### U-NET ARCHITECTURE

CREATING COSTUMIZED FORMULA TO USE DICE COEFFICIENT AS A METRICS

In [ ]:
def dice_coefficient(y_true, y_pred):  #FUNCTION TAKING GROUND TRUTH LABELS AND PREDICTED LABELS  (LABELS ARE 1 AND 0: BINARY SEGMENTATION MASKS)
    smooth = 1.   #SMALL COSTANT ADDED TO AVOID DIVIDING BY ZERO
    y_true_f = tf.keras.backend.flatten(y_true)  #FLATTENING LABELS TO 1 D ARRAY FOR BOTH TRUE AND PREDICTED
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f) #INTERSECTION OF TRUE AND PREDICTED LABELS
    return (2. * intersection + smooth) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + smooth)   #DICE COEFF FORMULA MEASURING OVERLAP BETWEEN TRUE AND PREDICTED LABELS



class DiceCoefficient(keras.metrics.Metric):  #CLASS INERITHING FROM  keras.metrics.Metric
    def __init__(self, name='dice_coefficient', **kwargs): #CLASS INITIALISATION AND EVENTUAL ARGUMENTS
        super(DiceCoefficient, self).__init__(name=name, **kwargs) #INITIALISER CALCULATION OF THE PARENT CLASS
        self.dice = self.add_weight(name='dc', initializer='zeros') #WEIGHT VARIABLE INITIALISED TO ZERO ADDED

    def update_state(self, y_true, y_pred, sample_weight=None):  #METHOD TO UPDATE STATE OF THE METRIC
        dice = dice_coefficient(y_true, y_pred) #CALCULATING DICE COEFF
        self.dice.assign(dice)  #UPDATING DICE WEIGHT WITH CALCUATED COEFF

    def result(self):
        return tf.cast(self.dice, tf.float32)  #RETURNING A DICE WEIGHT WITH THAT DATATYPE

    def reset_states(self):
        self.dice.assign(0.0) #RESETTING DICE TO 0



CREATING COSTUMIZED FORMULA TO USE JACCARD COEFFICIENT AS A METRICS

In [ ]:
def jaccard_coefficient(y_true, y_pred):  #SAME OPERATIONS THAN DICE BUT THIS TIME I DEFINE BOTHE INTERSECTION AND UNION SINCE I NEED BOTH IN THE FORMULA
    smooth = 1.
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    union = tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) - intersection
    return (intersection + smooth) / (union + smooth)

class JaccardCoefficient(keras.metrics.Metric):
    def __init__(self, name='jaccard_coefficient', **kwargs):
        super(JaccardCoefficient, self).__init__(name=name, **kwargs)
        self.jaccard = self.add_weight(name='jc', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        jaccard = jaccard_coefficient(y_true, y_pred)
        self.jaccard.assign(jaccard)

    def result(self):
        return tf.cast(self.jaccard, tf.float32)

    def reset_states(self):
        self.jaccard.assign(0.0)


BUILDING THE ARCHITECTURE WITH HYPERPARAMETERS CHOSEN TO BE TUNED

In [ ]:
class UNetHyperModel(HyperModel): #NEW CLASS INHERITING THE HYPERMODEL, A CLASS USED TO BUILD U NET WITH HYPERPARAMETERS TUNING
    def build(self, hp):
        inputs = keras.Input(shape=(256, 256,1)) #INPUT:GRAYSCALE IMAGES 256X256

        # Encoder: hyp tuned: NR LAYERS, FILTERS SIZE PER LAYER, KERNEL SIZE PER LAYER, DROPOUT
        x = inputs #initialisation of inputs to build the encoder
        for i in range(hp.Int('num_layers', 3, 5)):  #in the encoder there'll be between 2 and 5 layers(hyperparam num layers encoder) #and there'll be looping over them
            x = keras.layers.Conv2D(              #2 D conv layer added
                filters=hp.Int(f'filters_{i}', 32, 256, step=32),  #filters from 32 to 256 in steps of 32
                kernel_size=hp.Choice(f'kernel_size_{i}', [3, 5]), #kernel hyperparameter
                activation='relu',
                padding='same' #output must have same width and height than input
            )(x)
            x = keras.layers.Dropout(hp.Float(f'dropout_{i}', 0.0, 0.5, step=0.1))(x) #dropout other hyperparameter (OVERFITTING PREVENTION): from 0 to 0.5 in steps of 0.1: for every conv2d layer there'll be a dropout
            x = keras.layers.MaxPooling2D()(x) #maxpooling layer: for every conv2d layer there'll be one (TO REDUCE SPATIAL DIMENSION OF FEATURE MAPS)

        # Bottleneck #just one conv 2d added, with following hyperparamters to tune: filters (like before but with steps of 64)
        x = keras.layers.Conv2D(
            filters=hp.Int('bottleneck_filters', 256, 512, step=64),
            kernel_size=hp.Choice('bottleneck_kernel_size', [3, 5]),
            activation='relu',
            padding='same'
        )(x)

        # Decoder #same hp tuning strategy
        for i in range(hp.Int('num_layers', 3, 5)):
            x = keras.layers.Conv2DTranspose(
                filters=hp.Int(f'decoder_filters_{i}', 32, 256, step=32),
                kernel_size=hp.Choice(f'decoder_kernel_size_{i}', [3, 5]),
                activation='relu',
                padding='same'
            )(x)
            x = keras.layers.Dropout(hp.Float(f'decoder_dropout_{i}', 0.0, 0.5, step=0.1))(x)
            x = keras.layers.UpSampling2D()(x)  #upsampling TO INCREASE SPATIAL DIMENSIONS OF FEATURE MAPS

        outputs = keras.layers.Conv2D(  #FINAL CONV LAYER FOR THE OUTPUT
            filters=1,
            kernel_size=1,
            activation='sigmoid' #SIGMOID CAUSE I NEED PROBABILITIES, PROBABILITY FOR A PIXEL TO BE 0 OR 1
        )(x)

        model = keras.Model(inputs, outputs) #KERAS MODEL CREATED WITH THE SPECIFIED INPUTS AND OUTPUTS
        model.compile(
            optimizer=keras.optimizers.Adam(  #ADAM OPTIMIZER
                hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4]) #ANOTHER HYPERPARAMETER: LEARNING RATE
            ),
            loss='binary_crossentropy',  #BINARY SEGM MASKS: BINARY CROSS ENTROPY
            metrics=[JaccardCoefficient()] #MY METRIC USED TO EVALUATE THE MODEL
        )
        return model

BUILDING THE OBJECT AND THE TUNER

In [ ]:
hypermodel = UNetHyperModel() #INSTANCE CREATION FOR THE CLASS UNETHYPERMODEL PREVIOSULY CREATED

tuner = Hyperband( #HYPERBAND TUNER ALGORITHM
    hypermodel, #MODEL TO BE TUNED
    objective=Objective("val_jaccard_coefficient", direction="max"), #OBJECTIVE IS MAXIMIZING JACCARD COEFF
    max_epochs=100, #MAX EPOCH PER MODEL
    factor=5, #REDUCTION FACTOR
    directory='Work directory', #DIR TO SAVE
    project_name='unet_tuning'
)

early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)  #TRAINING WILL STOP IF THERE'RE 5 EPOCHS WITH NO IMPROVEMENT IN VALIDATION LOSS

APPLYING THE ALGORITHM TO MY DATA, TO FIND THE BEST COMBINATION OF HYPERPARAMETERS

In [ ]:
#I use checkpoint to stop the training everytime i want and I can get the best model till that point
checkpoint_callback = ModelCheckpoint(
    filepath='best_model.h5',  # Path where the model will be saved
    monitor='val_loss',        # Metric to monitor
    save_best_only=True,       # Save only the best model
    mode='min'                 # Mode for the monitored metric
)

In [ ]:
tuner.search(train_data_images, train_data_masks, epochs=100, batch_size=32, validation_data=(validation_data, validation_mask),  callbacks=[checkpoint_callback] )
#Hyperband tuning process: goal to find best hyperparameters combination by maximizing jaccard coeff

GETTING BEST MODEL

In [ ]:
best_model = tuner.get_best_models(num_models=1)[0]   #RETRIEVING ONE MODE, THE TOP 1 (FIRST FROM THE LIST): BEST MODEL FOUND DURING HYPER PARAM TUNING

LOADING THE BEST MODEL, TO USE IT AGAIN EVEN WHEN THIS SESSION IS OVER

In [ ]:
best_model = load_model('best_model.h5')

In [ ]:
best_model.summary() #detailed summary of the model’s architecture, including the layers, output shapes, and the number of parameters.

FITTING THE BEST MODEL

In [ ]:
best_model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=[JaccardCoefficient()]
)

In [ ]:
#combining train and validation data for fitting the best model
combined_data = np.concatenate((train_data_images, validation_data))
combined_masks = np.concatenate((train_data_masks, validation_mask))

In [ ]:
history=best_model.fit(combined_data, combined_masks, epochs=100)

PLOTTING THE JACCARD INDEX (OR OTHERE METRIC) OVER EPOCHS GRAPH

In [ ]:
# Plot the training and validation loss
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title('Loss Over Epochs')

# Plot the training and validation metric of my best model over epochs
plt.subplot(1, 2, 2)
plt.plot(history.history['jaccard_coefficient'], label='Training Jaccard Coefficient')
plt.plot(history.history['val_jaccard_coefficient'], label='Validation Jaccard Coefficient')
plt.legend()
plt.title('Jaccard Coefficient Over Epochs')
plt.show()

plt.show()

EVALUATING THE MODEL ON TEST DATA

In [ ]:
results = best_model.evaluate(test_data, test_mask)  #BEST MODEL EVALUATED ON TEST DATA
print(results)

VISUALIZING PREDICTED MASKS TO COMPARE THEM WITH THE INPUT ONES

In [ ]:
 #VISUALIZING PREDICTIONS ON TEST DATA AND COMPARING WITH THE REAL MASKS
def visualize_predictions(model, test_images, test_masks, num_images=5):
    predictions = model.predict(test_images)

    for i in range(num_images):
        plt.figure(figsize=(15, 5))

        # Original image
        plt.subplot(1, 3, 1)
        plt.imshow(test_images[i].squeeze(), cmap='gray')
        plt.title('Original Image')

        # True mask
        plt.subplot(1, 3, 2)
        plt.imshow(test_masks[i].squeeze(), cmap='gray')
        plt.title('True Mask')

        # Predicted mask
        plt.subplot(1, 3, 3)
        plt.imshow(predictions[i].squeeze(), cmap='gray')
        plt.title('Predicted Mask')

        plt.show()

# Example usage
visualize_predictions(best_model, test_data, test_mask)


VISUALIZING ALL OTHER RELEVANT METRICS FOR MY BEST MODEL

In [ ]:
predictions = best_model.predict(test_data)
accuracy = accuracy_score(test_mask, predictions)
precision = precision_score(test_mask, predictions)
recall = recall_score(test_mask, predictions)
f1 = f1_score(test_mask, predictions)
auc = roc_auc_score(test_mask, predictions)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')
print(f'AUC: {auc}')


CONFUSION MATRIX

In [ ]:
img_true = test_mask.flatten() #flattening is necessary cause we need a 1D array
img_pred = (predictions > 0.5).astype(int).flatten()   #predicted probabilities converted as binary values: i will end with a flattened array with 1 or 0 (int values)
cm = confusion_matrix(img_true, img_pred)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Background', 'Object'], yticklabels=['Background', 'Object'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix for Binary Segmentation')
plt.show()
